# HOW TO ADD NEW DATA

## Adding new hazards

In `src/physrisk/kernel/hazards.py`, all hazards are cataloged, classified as ACUTE, CHRONIC, or UNKNOWN, and designated as either parameter-based or event-based. To add a new hazard, create a new class within this file and specify its type.

Additionally, complete the onboarding process for the new hazard in the hazard program. This step ensures that the hazard and its data are collected in the bucket and included in the inventory used by PhysRisk to calculate impacts and risk measures.

## Adding new vulnerability models

In the `src/physrisk/vulnerability_models` folder, files correspond to each vulnerability model (e.g., `chronic_heat_models`, `power_generating_asset_models`, `real_estate_models`, `thermal_power_generation_models`). Each file contains classes for each type of hazard, with separate classes for default and stress test calculations. The `DictBasedVulnerabilityModelsFactory`, which inherits from the `VulnerabilityModelsFactory` class in `src/physrisk/kernel/vulnerability_model.py`, is used to handle the different vulnerability models, whether they are default or stress test models.

In the `DictBasedVulnerabilityModelsFactory` class, there is a `vulnerability_models` method that retrieves the corresponding vulnerability models using the methods `get_default_vulnerability_models` and `get_stress_test_vulnerability_models`, implemented in `src/physrisk/kernel/calculation.py`.

To add a vulnerability model for a new asset type, create a new file with the corresponding vulnerability classes for each hazard type. Additionally, create a JSON file in the `src/physrisk/datas/static/vulnerability` folder. This JSON file should include the vulnerability curves for these models, detailing `impact_mean`, `impact_std`, `impact_type`, `impact_units`, `intensity`, `intensity_units`, and `location` for each type of event (hazard) and asset.

For adding a vulnerability model for an existing asset, create a new class in the relevant file. All classes must inherit from either a class in `src/physrisk/kernel/vulnerability_model.py` or another class in the same file that inherits from a class in `vulnerability_model.py`. These classes must include at least a constructor, a `get_data_requests` method, and a `get_impact` method.

- The `get_data_requests` method returns an `HazardDataRequest` object, which stores all necessary information to request data from the bucket for calculating impacts and risk measures.
- The `get_impact` method returns an `ImpactDistrib` object, which contains "Impact distributions specific to an asset" and is used for calculating `impact_bins_explicit`, `mean_impact`, `stddev_impact`, `above_mean_stddev_impact`, and `to_exceedance_curve`.

To include the new vulnerability model, update either `get_default_vulnerability_models` or `get_stress_test_vulnerability_models` as appropriate. If introducing a new calculation method, add a new method and integrate it into `DictBasedVulnerabilityModelsFactory`. 

## Adding new risk models

In `src/physrisk/risk_models/risk_models.py`, there are three classes that implement risk models: `RealEstateToyRiskMeasures` (which calculates risk measures using exceedance curves), `ThermalPowerPlantsRiskMeasures` (which calculates risk measures using mean intensity and percentiles). Additionally, in `src/physrisk/risk_models/generic_risk_model.py`, the `GenericScoreBasedRiskMeasures` class showcases how different approaches can be combined for calculating risk scores, using vulnerability models for some hazards and direct hazard indicators for others. This generic implementation serves as an example, blending elements from both real estate and Jupiter exposure calculations, without a predefined use case for the measures.

Moreover, similar to the vulnerability models, a factory class `DefaultMeasuresFactory` has been implemented in `src/physrisk/kernel/calculation.py` to select the appropriate risk model based on the `use_case_id`. In this class, there is a method called `calculators` which makes use of `get_default_risk_measure_calculators`, `get_stress_test_risk_measure_calculators`, and `get_default_risk_measure_calculators`, implemented in `src/physrisk/kernel/calculation.py`.

To add new risk models, you need to create a new class in the risk_models file that implements the calculations for the new model.
